In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys
sys.path.insert(0, '../YOLO_intro')
from Object_Detection import ObjectDectection
from ObjectDetectionGenerator import ObjectDetectionGenerator
from yolo_helper import GeneratorMultipleOutputs

Using TensorFlow backend.


In [3]:
PASCAL_VOC_FOLDER = '/home/usuario/repos/Object_Localization/dataset/'
PASCAL_VOC_ANNOTATIONS_FOLDER = PASCAL_VOC_FOLDER + 'ILSVRC/Annotations/CLS-LOC/'
PASCAL_VOC_IMAGES_FOLDER = PASCAL_VOC_FOLDER + 'ILSVRC/Data/CLS-LOC/'

In [4]:
NUMBER_OF_CLASSES = 8
IMAGE_H = 320
IMAGE_W = 320
GRID_H = IMAGE_H // 32
GRID_W = IMAGE_W // 32
NUMBER_OF_BBOXES = 1
batch_size = 60
classes = ['n02165456', 'n02226429', 'n01644373', 'n02279972', 'n01632458', 'n02256656', 'n01774384', 'n01770393']

In [60]:
train_generator = ObjectDetectionGenerator(PASCAL_VOC_ANNOTATIONS_FOLDER, 
                                           PASCAL_VOC_IMAGES_FOLDER, 
                                           batch_size,
                                           (IMAGE_H,IMAGE_W),
                                           classes,
                                           NUMBER_OF_BBOXES = 1,
                                           PASCAL_VOC_dataset='train',
                                           GRID_H = GRID_H, GRID_W = GRID_W
                                          )

(1/8) Class n02165456 has 675 images
(2/8) Class n02226429 has 711 images
(3/8) Class n01644373 has 676 images
(4/8) Class n02279972 has 644 images
(5/8) Class n01632458 has 687 images
(6/8) Class n02256656 has 643 images
(7/8) Class n01774384 has 655 images
(8/8) Class n01770393 has 603 images
found 5294 images in 8 classes


In [61]:
# batch_size = 60
# reduced_dataset_folder = '/home/usuario/repos/Object_Localization/challenge_dataset_v3'
# train_folder = reduced_dataset_folder + '/images/train'
# annotations_dict_train = np.load(reduced_dataset_folder+'/annotations_train.npy').flat[0]
# train_generator = GeneratorMultipleOutputs(annotations_dict_train, 
#                             train_folder, batch_size, classes = classes,
#                             target_size=(IMAGE_H,IMAGE_W),
#                             GRID_H = GRID_H,
#                             GRID_W = GRID_W, 
#                             flip_horizontal = True, 
#                             flip_vertical = True,
#                             crop_per=0.2
#                            )

In [62]:
object_detection = ObjectDectection(NUMBER_OF_CLASSES, IMAGE_H, IMAGE_W, NUMBER_OF_BBOXES)

In [63]:
import tensorflow as tf
import keras.backend as K
from Object_Detection import Losses

losses = Losses(len(classes))

In [64]:
image, y_true = next(train_generator)

In [65]:
y_pred = object_detection.model.predict_on_batch(image)

In [75]:
y_true_tf = tf.constant(y_true, dtype='float32')
y_pred_tf = tf.constant(y_pred, dtype='float32')

box_mse_out = losses.bounding_box_mse()(y_true_tf, y_pred_tf)
cat_out = losses.categorical_cross_entropy_loss()(y_true_tf, y_pred_tf)
obj_out = losses.object_bin_cross_entropy_loss()(y_true_tf, y_pred_tf)
no_obj_out = losses.no_object_bin_cross_entropy_loss()(y_true_tf, y_pred_tf)
focal_loss = losses.focal_loss()(y_true_tf, y_pred_tf)
yolo_loss = losses.YOLO_loss()(y_true_tf, y_pred_tf)

In [77]:
sess = tf.Session()
with sess.as_default():
    print(box_mse_out.eval().shape)
    print(cat_out.eval().shape)
    print(obj_out.eval().shape)
    print(no_obj_out.eval().shape)
    print(focal_loss.eval().shape)
    print(yolo_loss.eval())
    
# 0.086642854
# 0.08664659

(60,)
(60,)
(60,)
(5940,)
(60,)
23.31504


In [28]:
y_true, y_pred

(array([[[[[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          ...,
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          ...,
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
      

In [105]:
def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/N
    return ce

predictions = np.array([[0.25]])
targets = np.array([[1]])
x = cross_entropy(predictions, targets)
print(x)

1.3862943571198905
